In [1]:
import pandas as pd
import glob
import regex as re

In [26]:
files = glob.glob("results/*.csv")
dfs = [pd.read_csv(f, index_col=0) for f in files]
files

['results/eval_results_DBpedia_oneshot_NousResearch-Hermes-3-Llama-3.2-3B-GGUF.csv',
 'results/eval_results_DBpedia_oneshot_NousResearch-Hermes-3-Llama-3.1-8B-GGUF.csv',
 'results/eval_results_DBpedia_zeroshot_NousResearch-Hermes-3-Llama-3.2-3B-GGUF.csv']

In [42]:
for f, df in zip(files, dfs):
    # data=re.findall(r"results/eval_results_(\w+)_.*", f)[0]
    # prompt_type=re.findall(r"results/eval_results_(\w+)_.*", f)[0]
    model = re.findall(r"results/eval_results_\w+_(.+)\.csv", f)[0]
    # df['prompt_type'] = prompt_type
    df["model"] = (
        model.replace("_", " ")
        .replace("-", " ")
        .rstrip("-GGUF")
        .lstrip("NousResearch Hermes 3")
    )

df = pd.concat(dfs)
df

erl  \
0     {"relations":[{"entity":"animal","relation":"b...   
1     {"relations":[{"entity":"musical artist","rela...   
2     {"relations":[{"entity":"musical work","relati...   
3     {"relations":[{"entity":"company","relation":"...   
4     {"relations":[{"entity":"person","relation":"c...   
...                                                 ...   
1795  {"relations":[{"entity":"ethnic group","relati...   
1796  {"relations":[{"entity":"Organisation member",...   
1797  {"relations":[{"entity":"Organisation member",...   
1798  {"relations":[{"entity":"person","relation":"b...   
1799  {"relations":[{"entity":"person","relation":"b...   

                                               response  generator  n_nodes  \
0     {"relations":[{"entity":"animal 1","relation":...      human        3   
1     {"relations":[{"entity":"artist 1","relation":...      human        3   
2     {"relations":[{"entity":"person 1","relation":...      human        3   
3     {"relations":[{"entity":"written work 1","rela...      human        5   
4     {"relations":[{"entity":"actor 1","relation":"...      human        5   
...                                                 ...        ...      ...   
1795  a ethnic group has a population place with pop...  templated       10   
1796  An Organisation member has nationality of a co...      llama       10   
1797  a Organisation member has a nationality with c...  templated       10   
1798  - A person is born in a populated place.\n- A ...      llama       10   
1799  a person has a birth place with populated plac...  templated       10   

      seed                                         erl_loaded  f1_score  \
0       60  relations=[EnrichedRelation(entity='animal', r...  1.000000   
1       12  relations=[EnrichedRelation(entity='musical ar...  0.666667   
2      157  relations=[EnrichedRelation(entity='musical wo...  0.400000   
3      134  relations=[EnrichedRelation(entity='company', ...  1.000000   
4      121  relations=[EnrichedRelation(entity='person', r...  1.000000   
...    ...                                                ...       ...   
1795   297  relations=[EnrichedRelation(entity='ethnic gro...       NaN   
1796   298  relations=[EnrichedRelation(entity='Organisati...       NaN   
1797   298  relations=[EnrichedRelation(entity='Organisati...       NaN   
1798   299  relations=[EnrichedRelation(entity='person', r...       NaN   
1799   299  relations=[EnrichedRelation(entity='person', r...       NaN   

      precision    recall  edit_distance          model cfg_name zeroshot  
0           1.0  1.000000       1.000000  Llama 3.2 3B   DBpedia    False  
1           1.0  0.500000       0.333333  Llama 3.2 3B   DBpedia    False  
2           0.5  0.333333       0.200000  Llama 3.2 3B   DBpedia    False  
3           1.0  1.000000       1.000000  Llama 3.2 3B   DBpedia    False  
4           1.0  1.000000       0.166667  Llama 3.2 3B   DBpedia    False  
...         ...       ...            ...            ...      ...      ...  
1795        NaN       NaN            NaN  Llama 3.2 3B       NaN      NaN  
1796        NaN       NaN            NaN  Llama 3.2 3B       NaN      NaN  
1797        NaN       NaN            NaN  Llama 3.2 3B       NaN      NaN  
1798        NaN       NaN            NaN  Llama 3.2 3B       NaN      NaN  
1799        NaN       NaN            NaN  Llama 3.2 3B       NaN      NaN  

[5427 rows x 13 columns]

In [43]:
df["prompt_type"] = df["zeroshot"].apply(lambda x: "zero shot" if x else "one shot")
df.drop(columns=["zeroshot"], inplace=True)
df

,erl,response,generator,n_nodes,seed,erl_loaded,f1_score,precision,recall,edit_distance,model,cfg_name,prompt_type
0,"{""relations"":[{""entity"":""animal"",""relation"":""b...","{""relations"":[{""entity"":""animal 1"",""relation"":...",human,3,60,"relations=[EnrichedRelation(entity='animal', r...",1.000000,1.0,1.000000,1.000000,Llama 3.2 3B,DBpedia,one shot
1,"{""relations"":[{""entity"":""musical artist"",""rela...","{""relations"":[{""entity"":""artist 1"",""relation"":...",human,3,12,relations=[EnrichedRelation(entity='musical ar...,0.666667,1.0,0.500000,0.333333,Llama 3.2 3B,DBpedia,one shot
2,"{""relations"":[{""entity"":""musical work"",""relati...","{""relations"":[{""entity"":""person 1"",""relation"":...",human,3,157,relations=[EnrichedRelation(entity='musical wo...,0.400000,0.5,0.333333,0.200000,Llama 3.2 3B,DBpedia,one shot
3,"{""relations"":[{""entity"":""company"",""relation"":""...","{""relations"":[{""entity"":""written work 1"",""rela...",human,5,134,"relations=[EnrichedRelation(entity='company', ...",1.000000,1.0,1.000000,1.000000,Llama 3.2 3B,DBpedia,one shot
4,"{""relations"":[{""entity"":""person"",""relation"":""c...","{""relations"":[{""entity"":""actor 1"",""relation"":""...",human,5,121,"relations=[EnrichedRelation(entity='person', r...",1.000000,1.0,1.000000,0.166667,Llama 3.2 3B,DBpedia,one shot
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1795,"{""relations"":[{""entity"":""ethnic group"",""relati...",a ethnic group has a population place with pop...,templated,10,297,relations=[EnrichedRelation(entity='ethnic gro...,NaN,NaN,NaN,NaN,Llama 3.2 3B,NaN,zero shot
1796,"{""relations"":[{""entity"":""Organisation member"",...",An Organisation member has nationality of a co...,llama,10,298,relations=[EnrichedRelation(entity='Organisati...,NaN,NaN,NaN,NaN,Llama 3.2 3B,NaN,zero shot
1797,"{""relations"":[{""entity"":""Organisation member"",...",a Organisation member has a nationality with c...,templated,10,298,relations=[EnrichedRelation(entity='Organisati...,NaN,NaN,NaN,NaN,Llama 3.2 3B,NaN,zero shot
1798,"{""relations"":[{""entity"":""person"",""relation"":""b...",- A person is born in a populated place.\n- A ...,llama,10,299,"relations=[EnrichedRelation(entity='person', r...",NaN,NaN,NaN,NaN,Llama 3.2 3B,NaN,zero shot


In [44]:
df_clean = df.dropna()
df_clean.describe()

,n_nodes,seed,f1_score,precision,recall,edit_distance
count,2847.000000,2847.000000,2847.000000,2847.000000,2847.000000,2847.000000
mean,5.687741,147.968388,0.746787,0.874661,0.697876,0.494398
std,2.830873,85.708734,0.258802,0.204950,0.291957,0.341155
min,3.000000,0.000000,0.133333,0.200000,0.100000,-0.350000
25%,3.000000,74.000000,0.500000,0.750000,0.500000,0.222222
50%,5.000000,147.000000,0.800000,1.000000,0.700000,0.500000
75%,10.000000,222.000000,1.000000,1.000000,1.000000,0.794737
max,10.000000,299.000000,1.000000,1.000000,1.000000,1.000000


In [46]:
means = df_clean.groupby(["n_nodes", "prompt_type", "model", "generator", "cfg_name"])[
    ["f1_score", "edit_distance"]
].mean()
counts = df_clean.groupby(["n_nodes", "prompt_type", "model", "generator", "cfg_name"])[
    ["f1_score", "edit_distance"]
].count()
means

f1_score  edit_distance
n_nodes prompt_type model         generator cfg_name                         
3       one shot    Llama 3.1 8B  human     DBpedia   0.700000       0.500000
                                  llama     DBpedia   0.773518       0.564611
                                  templated DBpedia   0.873156       0.737290
                    Llama 3.2 3B  human     DBpedia   0.688889       0.511111
                                  llama     DBpedia   0.792252       0.535848
                                  templated DBpedia   0.879489       0.649780
        zero shot   Llama 3.2 3B  human     DBpedia   0.500000       0.200000
                                  llama     DBpedia   0.648958       0.376246
                                  templated DBpedia   0.608712       0.428869
5       one shot    Llama 3.1 8B  human     DBpedia   0.933333       0.700000
                                  llama     DBpedia   0.733572       0.442061
                                  templated DBpedia   0.845764       0.640515
                    Llama 3.2 3B  human     DBpedia   1.000000       0.722222
                                  llama     DBpedia   0.716179       0.419280
                                  templated DBpedia   0.797523       0.523367
        zero shot   Llama 3.2 3B  human     DBpedia   0.500000       0.194444
                                  llama     DBpedia   0.563743       0.303331
                                  templated DBpedia   0.566283       0.466193
10      one shot    Llama 3.1 8B  human     DBpedia   0.652288       0.232080
                                  llama     DBpedia   0.666543       0.285583
                                  templated DBpedia   0.793400       0.566343
                    Llama 3.2 3B  human     DBpedia   0.583333       0.148962
                                  llama     DBpedia   0.608290       0.295760
                                  templated DBpedia   0.762484       0.497377
        zero shot   Llama 3.2 3B  human     DBpedia   0.449495       0.323810
                                  llama     DBpedia   0.390524       0.089775
                                  templated DBpedia   0.530473       0.288700

In [49]:
counts

f1_score  edit_distance
n_nodes prompt_type model         generator cfg_name                         
3       one shot    Llama 3.1 8B  human     DBpedia          2              2
                                  llama     DBpedia        213            213
                                  templated DBpedia        215            215
                    Llama 3.2 3B  human     DBpedia          3              3
                                  llama     DBpedia        244            244
                                  templated DBpedia        244            244
        zero shot   Llama 3.2 3B  human     DBpedia          1              1
                                  llama     DBpedia         64             64
                                  templated DBpedia         40             40
5       one shot    Llama 3.1 8B  human     DBpedia          3              3
                                  llama     DBpedia        208            208
                                  templated DBpedia        210            210
                    Llama 3.2 3B  human     DBpedia          3              3
                                  llama     DBpedia        230            230
                                  templated DBpedia        231            231
        zero shot   Llama 3.2 3B  human     DBpedia          2              2
                                  llama     DBpedia         79             79
                                  templated DBpedia         53             53
10      one shot    Llama 3.1 8B  human     DBpedia          3              3
                                  llama     DBpedia        181            181
                                  templated DBpedia        171            171
                    Llama 3.2 3B  human     DBpedia          3              3
                                  llama     DBpedia        169            169
                                  templated DBpedia        157            157
        zero shot   Llama 3.2 3B  human     DBpedia          3              3
                                  llama     DBpedia         68             68
                                  templated DBpedia         47             47

In [94]:
column_map = {
    "f1_score": "$F_1$",
    "edit_distance": "$GED_{s}$",
    "prompt_type": "Prompt Type",
    "model": "Model",
    "n_nodes": "$m_{\\text{nodes}}$",
    "generator": "Query Origin",
}

## Compare Query Origins on DBpedia with 8B model

In [ ]:
query_gen_compare = df_clean[
    df_clean["model"].str.contains("8B") & df_clean["cfg_name"].str.contains("DBpedia") & df_clean["prompt_type"].str.contains("one shot")
]
grouped_query_gen_compare = query_gen_compare.groupby(["n_nodes", "generator"])[
    ["f1_score", "edit_distance"]
].mean()
grouped_query_gen_compare = grouped_query_gen_compare.rename(
    columns=column_map
).rename_axis([column_map["n_nodes"], column_map["generator"]], axis=0)
with open("tables/compare_gen_query.tex", "w") as f:
    f.write(
        grouped_query_gen_compare.to_latex(
            float_format="%.2f",
            caption="Comparison of query generation methods for LLama 3.1 8B on DBpedia with one shot prompts.",
            label="tab:compare_gen_query",
        )
    )
grouped_query_gen_compare

$F_1$  $GED_{norm.}$
$n_{nodes}$ Query Origin                         
3           human         0.700000       0.500000
            llama         0.773518       0.564611
            templated     0.873156       0.737290
5           human         0.933333       0.700000
            llama         0.733572       0.442061
            templated     0.845764       0.640515
10          human         0.652288       0.232080
            llama         0.666543       0.285583
            templated     0.793400       0.566343

## Compare models across Datasets

In [96]:
datasets = ["DBpedia"]  # , "UniProt"]
for dataset in datasets:
    model_compare = df_clean[df_clean["cfg_name"].str.strip() == dataset]
    grouped_model_compare = model_compare.groupby(["n_nodes", "prompt_type", "model"])[
        ["f1_score", "edit_distance"]
    ].mean()
    grouped_model_compare = grouped_model_compare.rename(
        columns=column_map
    ).rename_axis([column_map["n_nodes"], column_map["prompt_type"], column_map["model"]], axis=0)
    with open(f"tables/compare_model_{dataset}.tex", "w") as f:
        f.write(
            grouped_model_compare.to_latex(
                float_format="%.2f",
                caption=f"Comparison of models for {dataset}",
                label=f"tab:compare_model_{dataset}",
            )
        )
grouped_model_compare

$F_1$  $GED_{norm.}$
$n_{nodes}$ Prompt Type Model                                 
3           one shot    Llama 3.1 8B   0.822995       0.650650
                        Llama 3.2 3B   0.834972       0.592314
            zero shot   Llama 3.2 3B   0.632208       0.394615
5           one shot    Llama 3.1 8B   0.790958       0.542891
                        Llama 3.2 3B   0.758511       0.473058
            zero shot   Llama 3.2 3B   0.563796       0.366121
10          one shot    Llama 3.1 8B   0.727528       0.420370
                        Llama 3.2 3B   0.681644       0.390634
            zero shot   Llama 3.2 3B   0.447766       0.174958

In [97]:
dataset

'DBpedia'

In [98]:
df_clean["cfg_name"].str.strip() == "DBpedia"

0       True
1       True
2       True
3       True
4       True
        ... 
1774    True
1787    True
1788    True
1789    True
1793    True
Name: cfg_name, Length: 2847, dtype: bool